<a href="https://colab.research.google.com/github/sadrovermonks/gmcodes/blob/main/gmappendstories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import bigquery

# Inicializa cliente con el project ID correcto
client = bigquery.Client(project='general-motors-global')

# Configuración del entorno
dataset_id = '099_gm_namer_test'
table_source = 'julytoappend_instagramstories'
table_dest = 'historical_instagramstories'

# Esquema destino (todo en STRING)
dest_schema = [
    {"name": "Post_ID", "type": "STRING"},
    {"name": "Account_ID", "type": "STRING"},
    {"name": "Account_username", "type": "STRING"},
    {"name": "Account_name", "type": "STRING"},
    {"name": "Description", "type": "STRING"},
    {"name": "Duration__sec_", "type": "STRING"},
    {"name": "Publish_time", "type": "STRING"},
    {"name": "Permalink", "type": "STRING"},
    {"name": "Post_type", "type": "STRING"},
    {"name": "Data_comment", "type": "STRING"},
    {"name": "Date", "type": "STRING"},
    {"name": "Views", "type": "STRING"},
    {"name": "Reach", "type": "STRING"},
    {"name": "Likes", "type": "STRING"},
    {"name": "Shares", "type": "STRING"},
    {"name": "Follows", "type": "STRING"},
    {"name": "Profile_visits", "type": "STRING"},
    {"name": "Link_clicks", "type": "STRING"},
    {"name": "Sticker_taps", "type": "STRING"},
    {"name": "Navigation", "type": "STRING"},
    {"name": "Replies", "type": "STRING"},
]

# Obtener esquema del origen
dataset_ref = f"{client.project}.{dataset_id}"
table_ref = f"{dataset_ref}.{table_source}"

dataset_obj = client.get_dataset(dataset_ref)
table = client.get_table(table_ref)
source_schema = table.schema

# Crear set de columnas del origen
source_cols = {field.name for field in source_schema}

# Preparar las columnas para la inserción
columns_to_insert = []

for col in dest_schema:
    col_name = col['name']
    if col_name in source_cols:
        # Solo casteamos a STRING
        cast_expr = f"CAST({col_name} AS STRING)"
        columns_to_insert.append(cast_expr)

# Construir las cadenas para la consulta final
columns_list_str = ", ".join([col['name'] for col in dest_schema])
select_exprs = ", ".join(columns_to_insert)

# Crear la consulta SQL
final_query = f"""
INSERT INTO `{client.project}.{dataset_id}.{table_dest}` ({columns_list_str})
SELECT {select_exprs}
FROM `{client.project}.{dataset_id}.{table_source}`
"""

# Ejecutar
query_job = client.query(final_query)
query_job.result()

print("Append en STRING completado exitosamente.")

Append en STRING completado exitosamente.
